Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [10]:
from random import choice, randint, random
from dataclasses import dataclass
from copy import copy
import sys
from tqdm import tqdm

import lab9_lib

## Problem Initialization

In [13]:
POPULATION_SIZE = 100
OFFSPRING_SIZE = 200
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .2
MUTATIONS = 1
LOCI = 1000
PROBLEM = 1
GENERATIONS = 10000
MIGRANTS = 10
ISLANDS = 4
MIGRATE = 100

fitness = lab9_lib.make_problem(PROBLEM)

In [7]:
@dataclass
class Individual:
    fitness: float
    genotype: list[bool]

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = [randint(0, LOCI-1) for _ in range(MUTATIONS)]
    for i in range(len(pos)):
        offspring.genotype[pos[i]] = not offspring.genotype[pos[i]]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, LOCI-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == LOCI
    return offspring

def uniform_crossover(ind1: Individual, ind2: Individual) -> Individual:
    offspring = Individual(fitness=None,
                           genotype=[choice([ind1.genotype[i], ind2.genotype[i]])
                                     for i in range(LOCI)])
    assert len(offspring.genotype) == LOCI
    return offspring

In [8]:
class Island:
    population: list[Individual]
    id: int

    def __init__(self, id):
        self.population = [
            Individual(
                genotype=[choice((True, False)) for _ in range(LOCI)],
                fitness=None,
            )
            for _ in range(POPULATION_SIZE)
        ]
        self.id = id

        for i in self.population:
            i.fitness = fitness(i.genotype)

    def step(self, best_fitness) -> float:
        for _ in range(MIGRATE):
            offspring = list()
            for _ in range(OFFSPRING_SIZE):
                if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
                    # mutation  # add more clever mutations
                    p = select_parent(self.population)
                    o = mutate(p)
                else:
                    # xover # add more xovers
                    p1 = select_parent(self.population)
                    p2 = select_parent(self.population)
                    # o = one_cut_xover(p1, p2)
                    o = uniform_crossover(p1, p2)
                offspring.append(o)

            for i in offspring:
                i.fitness = fitness(i.genotype)
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:POPULATION_SIZE]
    
            if self.population[0].fitness > best_fitness:
                best_fitness = self.population[0].fitness
                print("Island ", self.id, ": ", best_fitness)
        return best_fitness

    def inject(self, migrants: list[Individual]):
        self.population.extend(migrants)

    def migrate(self) -> list[Individual]:
        migrants = sorted(self.population, key=lambda x: x.fitness, reverse=True)[:MIGRANTS]
        return migrants

In [63]:
# fitness = lab9_lib.make_problem(PROBLEM)
'''for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

population = [
    Individual(
        genotype=[choice((True, False)) for _ in range(LOCI)],
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)'''

'for n in range(10):\n    ind = choices([0, 1], k=50)\n    print(f"{\'\'.join(str(g) for g in ind)}: {fitness(ind):.2%}")\n\nprint(fitness.calls)\n\npopulation = [\n    Individual(\n        genotype=[choice((True, False)) for _ in range(LOCI)],\n        fitness=None,\n    )\n    for _ in range(POPULATION_SIZE)\n]\n\nfor i in population:\n    i.fitness = fitness(i.genotype)'

## GA execution

In [1]:
'''best_fitness = 0
for generation in tqdm(range(GENERATIONS), file=sys.stdout):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = mutate(p)
        else:
            # xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i.genotype)
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    
    if population[0].fitness > best_fitness:
        best_fitness = population[0].fitness
        print(" ", best_fitness)'''
    

'best_fitness = 0\nfor generation in tqdm(range(GENERATIONS), file=sys.stdout):\n    offspring = list()\n    for counter in range(OFFSPRING_SIZE):\n        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability\n            # mutation  # add more clever mutations\n            p = select_parent(population)\n            o = mutate(p)\n        else:\n            # xover # add more xovers\n            p1 = select_parent(population)\n            p2 = select_parent(population)\n            o = one_cut_xover(p1, p2)\n        offspring.append(o)\n\n    for i in offspring:\n        i.fitness = fitness(i.genotype)\n    population.extend(offspring)\n    population.sort(key=lambda i: i.fitness, reverse=True)\n    population = population[:POPULATION_SIZE]\n    \n    if population[0].fitness > best_fitness:\n        best_fitness = population[0].fitness\n        print(" ", best_fitness)'

## The Island method

In [18]:
islands = []
for id in range(ISLANDS):
    i = Island(id)
    islands.append(i)

finished = False
best_fitness = 0
g = 0
while g < GENERATIONS // MIGRATE:
    print("STEP: ", g)
    for i in range(ISLANDS):
        if finished: 
            break
        print("ISLAND: ", i)
        best_fitness = islands[i].step(best_fitness)
        if best_fitness == 1.0: 
            finished = True
    migrants = []
    if finished:
        break
    for i in range(ISLANDS):
        migrants.append(islands[i].migrate())
    for j in range(ISLANDS):
        for k in range(ISLANDS):
            if j == k: continue
            islands[j].inject(migrants[k])
    g+=1
print("Finished!")


STEP:  0
ISLAND:  0
Island  0 :  0.543
Island  0 :  0.558
Island  0 :  0.576
Island  0 :  0.593
Island  0 :  0.619
Island  0 :  0.62
Island  0 :  0.632
Island  0 :  0.637
Island  0 :  0.648
Island  0 :  0.661
Island  0 :  0.669
Island  0 :  0.687
Island  0 :  0.691
Island  0 :  0.711
Island  0 :  0.715
Island  0 :  0.739
Island  0 :  0.744
Island  0 :  0.75
Island  0 :  0.769
Island  0 :  0.77
Island  0 :  0.778
Island  0 :  0.788
Island  0 :  0.791
Island  0 :  0.798
Island  0 :  0.811
Island  0 :  0.815
Island  0 :  0.822
Island  0 :  0.828
Island  0 :  0.834
Island  0 :  0.842
Island  0 :  0.845
Island  0 :  0.85
Island  0 :  0.861
Island  0 :  0.862
Island  0 :  0.868
Island  0 :  0.877
Island  0 :  0.88
Island  0 :  0.889
Island  0 :  0.89
Island  0 :  0.897
Island  0 :  0.901
Island  0 :  0.906
Island  0 :  0.908
Island  0 :  0.913
Island  0 :  0.92
Island  0 :  0.923
Island  0 :  0.925
Island  0 :  0.928
Island  0 :  0.932
Island  0 :  0.934
Island  0 :  0.939
Island  0 :  0.942